In [ ]:
!pip install pythainlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 47.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from pythainlp.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [ ]:
pos_text = ("/content/Positive.csv")
neg_text = ("/content/Negative.csv")
neu_text = ("/content/Neutrally.csv")

In [ ]:
pos_df = pd.read_csv(pos_text, delimiter=',', encoding='utf-8').dropna()
neg_df = pd.read_csv(neg_text, delimiter=',', encoding='utf-8').dropna()
neu_df = pd.read_csv(neu_text, delimiter=',', encoding='utf-8').dropna()

data = pd.concat([pos_df, neg_df, neu_df])
data.sample(100)

,User,Message,Sentiment
434,*******405,ไม่ตรงปก,Negative
424,K***.,ส่งเร็วค่ะ ผ้านิ่มลื่นบาง แต่ลองใส่แล้วมันรั้ง...,Neutrally
87,พ***.,คุณภาพของสินค้าดีมาก ความคุ้มค่าต่อราคาดีมาก ค...,Positive
1074,gunwirin A.,ดีมากๆๆ,Positive
367,Noppakoon19,ได้รับสินค้าแล้วนะถูกใจใหญ่มากค่ะเสียอย่างเดีย...,Positive
...,...,...,...
324,T***m,การรอสินค้าพอประมาณไม่ช้าไม่เร็ว ทางร้านแพ็คขอ...,Positive
1220,อรอุมา พ.,ผ้าดีค่ะถึงจะบางแต่ดีมากค่ะสวย,Negative
1085,Jan J.,เก็บความเย็นดีแต่ตะแคงแล้วน้ำหกครับ,Positive
1317,จ***.,การทำงานของร้านค้าช้ามาก กว่าจะส่งของช้ามากๆ แย่,Negative


In [ ]:
# สร้างฟังก์ชันสำหรับการลบอักขระที่ซ้ำกันในแต่ละคำเท่านั้น
def remove_duplicate_chars(text):
    if isinstance(text, str):  # ตรวจสอบว่าข้อความไม่ใช่ NaN
        unique_words = []
        words = word_tokenize(text)
        for word in words:
            unique_word = ''
            for char in word:
                if char not in unique_word:
                    unique_word += char
            unique_words.append(unique_word)
        return ' '.join(unique_words)
    else:
        return text  # ส่งค่า NaN กลับหากเป็น NaN

# ใช้ฟังก์ชัน `remove_duplicate_chars` เพื่อ Tokenize และลบคำที่ซ้ำ
data['Message'] = data['Message'].apply(remove_duplicate_chars)
data.sample(10)

,User,Message,Sentiment
1006,Mongkhol L.,จัดส่ง รวดเ็ หีบ่อ อย่าง ดี จุ ไฟ ได้ ...,Positive
531,*******359,เข้าใจ ยัด สี อื่น มา ให้า พ่อ ง ง ง ง,Negative
190,ลมเปลี่ยนทิศ แ.,ผ้ ไม่ ดี เลย ไม่ เค ร,Negative
67,T***g,ตอนแรก ลังเ ไม่ มี ใคร คอม เม้น เลย ลอง สั่ง...,Negative
111,*******393,ดี ค่ะ ะ,Positive
324,NARES P.,คุณภาพ ตาม ราค,Neutrally
1280,J***.,สั่ง สีน้ำเงิ สี ดำ แต่ ได้ สี ดำ ตัว,Neutrally
1215,????? ?.,ชาต ไม่ ถึง นาที แบต หมด ง มาก ซื้อ มา อัน ใ...,Negative
764,SOMPOJ T.,ห่วยแตก สั่ง ชมพู ส่ง เหลือง สั่ง ฟ้า ส่ง สี ฟ...,Negative
917,****5,รวม ๆ ดี,Neutrally


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

Message = data['Message'].values
Sentiment = data['Sentiment'].values

# Convert text to numerical features
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(Message).toarray()

# Encoding labels
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(Sentiment)

print("Encoded labels:", encoded_labels)
print("Mapping of encoded labels to original labels:")
for label, original_label in enumerate(encoder.classes_):
    print(f"{label}: {original_label}")

Encoded labels: [2 2 2 ... 1 1 1]
Mapping of encoded labels to original labels:
0: Negative
1: Neutrally
2: Positive


In [ ]:
# แบ่งข้อมูลเป็นชุดฝึกและชุดทดสอบ
X_train, X_test, y_train, y_test = train_test_split(X, encoded_labels, stratify=encoded_labels, test_size=0.1, random_state=42)

In [ ]:
def grid_search_hyperparameters(X_train, y_train, X_test, y_test, hyperparameters):
    best_accuracy = 0
    best_hyperparameters = None

    for params in hyperparameters:
        learning_rate, lambda_param, num_epochs, batch_size = params

        # Initialize and train the SVM model with the current hyperparameters
        svm_model = SVM(learning_rate=learning_rate, lambda_param=lambda_param, num_epochs=num_epochs, batch_size=batch_size)
        svm_model.fit(X_train, y_train)

        # Make predictions on the test set
        predictions = svm_model.predict(X_test)

        # Calculate accuracy
        accuracy = accuracy_score(y_test, predictions)

        # Update the best hyperparameters if the current ones perform better
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_hyperparameters = params

    return best_hyperparameters, best_accuracy


In [ ]:
learning_rates = [0.0001, 0.001, 0.01]
lambda_params = [0.001, 0.01, 0.1]
num_epochs_values = [200, 500, 1000]
batch_sizes = [16, 32, 64]
hyperparameters_grid = [(lr, lp, ne, bs) for lr in learning_rates
                                        for lp in lambda_params
                                        for ne in num_epochs_values
                                        for bs in batch_sizes]
best_hyperparameters, best_accuracy = grid_search_hyperparameters(X_train, y_train, X_test, y_test, hyperparameters_grid)

KeyboardInterrupt: 

In [ ]:
print("Best hyperparameters:", best_hyperparameters)
print("Best accuracy:", best_accuracy)

In [ ]:
class SVM:
    def __init__(self, learning_rate=0.0001, lambda_param=0.01, num_epochs=500, batch_size=32):
        self.learning_rate = learning_rate
        self.lambda_param = lambda_param
        self.num_epochs = num_epochs
        self.weights = None
        self.bias = None
        self.batch_size = batch_size

    def fit(self, X, y):
        num_samples, num_features = X.shape
        num_classes = len(np.unique(y))
        self.weights = np.zeros((num_features, num_classes))
        self.bias = np.zeros(num_classes)

        for _ in range(self.num_epochs):
            for batch_start in range(0, num_samples, self.batch_size):
                batch_end = min(batch_start + self.batch_size, num_samples)
                X_batch = X[batch_start:batch_end]
                y_batch = y[batch_start:batch_end]

                for idx, x_i in enumerate(X_batch):
                    for class_idx in range(num_classes):
                        y_val = 1 if y_batch[idx] == class_idx + 1 else -1
                        condition = y_val * (np.dot(x_i, self.weights[:, class_idx]) - self.bias[class_idx]) >= 1
                        if not condition:
                            self.weights[:, class_idx] -= self.learning_rate * (2 * self.lambda_param * self.weights[:, class_idx] - np.dot(x_i, y_val))
                            self.bias[class_idx] -= self.learning_rate * y_val

    def predict(self, X):
        approx = np.dot(X, self.weights) - self.bias
        return np.argmax(approx, axis=1) + 1

In [ ]:
# Train the SVM model
clf = SVM()
clf.fit(X_train, y_train)

In [ ]:
predictions = clf.predict(X_test)

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.52


In [ ]:
# Preprocess the input text
test_text_processed = remove_duplicate_chars("การทำงานของร้านค้าช้ามาก กว่าจะส่งของช้ามากๆ")

# Transform the preprocessed text into TF-IDF numerical features
test_text_tfidf = tfidf_vectorizer.transform([test_text_processed]).toarray()

# Predict the sentiment label using the trained SVM classifier
predicted_label = clf.predict(test_text_tfidf)

# Mapping of predicted label to sentiment
sentiment_mapping = {0: "Negative", 1: "Neutral", 2: "Positive"}

# Print the classification result
print("The text is classified as:", sentiment_mapping[predicted_label[0]])


The text is classified as: Positive
